In [ ]:
{-# LANGUAGE FlexibleContexts #-}

import Control.Monad.ST
import Control.Monad.Primitive
import Data.Vector.Generic.Base
import qualified Data.Vector.Mutable as MV
import qualified Data.Vector.Generic as MV
import System.Random.MWC

In [ ]:
:info runST

In [ ]:
repeatSwap :: MV.MVector s a -> Gen (PrimState (ST s)) -> Int ->  MV.MVector s a
repeatSwap v g 0 = return v
repeatSwap v g n = do
    let n' = n - 1
    i <- uniformR (0, MV.basicLength v) g
    MV.swap v i n'
    repeatSwap v g n'

In [ ]:
body :: [a] -> Gen (PrimState (ST s)) -> ST s [a]
body xs g = do
    v <- MV.thaw $ MV.fromList xs
    repeatSwap v g $ length v
    MV.toList <$> MV.freeze v

In [ ]:
randomST :: [a] -> IO [a]
randomST xs = withSystemRandom . asGenST $ body xs

In [ ]:
shuffleList :: (Show a, NFData a) => [a] -> [a]
shuffleList (a:[]) = [a]
shuffleList xs =  runST $ withSystemRandom . asGenST $ \g -> do
    v <- MV.thaw $ MV.fromList xs
    v' <- modify v g n
    r <- MV.toList <$> MV.freeze v'
    return r
        where
            n = length xs
            modify v g 0 = return v
            modify v g i = do
                let i' = i - 1
                j <- uniformR (0,n) g
                MV.swap v i' j
                modify v g i'